# Рекомендательные системы. Гибридные рекомендательные системы.

> На этом практическом занятии мы с вами сделаем следующее:
- Посмотри как работает LightFM.
- Сравним качество и скорость BRP и WARP.
- Построим простейший сервис для рекомендаций используя LightFM.

## Импортируем библиотеку LightFM

In [5]:
conda install -c conda-forge/label/cf201901 lightfm


ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.

In [2]:
import numpy as np
import pandas as pd
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.datasets import fetch_movielens

from flask import Flask, jsonify, request

ModuleNotFoundError: No module named 'lightfm'

In [ ]:
movielens = fetch_movielens()
train, test = movielens['train'], movielens['test']

In [ ]:
train.todense()

## Сравним качество BRP и WARP

Зададим общие гиперпараметры

In [ ]:
alpha = 1e-05
epochs = 70
num_components = 32

In [ ]:
warp_model = LightFM(no_components=num_components,
                    loss='warp',
                    learning_schedule='adagrad',
                    max_sampled=100,
                    user_alpha=alpha,
                    item_alpha=alpha)

In [ ]:
bpr_model = LightFM(no_components=num_components,
                    loss='bpr',
                    learning_schedule='adagrad',
                    user_alpha=alpha,
                    item_alpha=alpha)

In [ ]:
logistic_model = 
#################
# YOUR CODE HERE
#################

In [ ]:
warp_duration = []
bpr_duration = []
logistic_duration = []
warp_auc = []
bpr_auc = []
logistic_auc = []

In [ ]:
for epoch in range(epochs):
    start = time.time()
    warp_model.fit_partial(train, epochs=1)
    warp_duration.append(time.time() - start)
    warp_auc.append(auc_score(warp_model, test, train_interactions=train).mean())

for epoch in range(epochs):
    start = time.time()
    bpr_model.fit_partial(train, epochs=1)
    bpr_duration.append(time.time() - start)
    bpr_auc.append(auc_score(bpr_model, test, train_interactions=train).mean())
    
#################
# YOUR CODE HERE
#################

#### AUC

Посмотрим на результаты обучения. Видно, что WARP гораздо точнее BRP. Видно также, что после 10 эпох качество снижается - модель начинает переобучаться.

In [ ]:
x = np.arange(epochs)
plt.figure(figsize = (12, 8))
plt.plot(x, np.array(warp_auc))
plt.plot(x, np.array(bpr_auc))
#################
# YOUR CODE HERE
#################
plt.legend(['WARP AUC', 'BPR AUC' , 'Logistic AUC'], loc='upper right') #
plt.show()

#### Скорость

BRP обучается быстрее. Видно также, что чем ближе к концу обучения, тем WARP обучается дольше. Это связано с тем, что чем лучше модель обучилась, тем больше надо перебрать негативных объектов, чтобы найти пример нарушения ранжирования.

> max_sampled гиперпарамеетр в WARP отвечает за максимальное количество попыток найти нарушение ранжирования.

In [ ]:
x = np.arange(epochs)
plt.figure(figsize = (12, 8))
plt.plot(x, np.array(warp_duration))
plt.plot(x, np.array(bpr_duration))
#################
# YOUR CODE HERE
#################
plt.legend(['WARP duration', 'BPR duration', 'Logistic duration'], loc='upper right')
plt.show()

## По традиции запустим сервис рекомендаций

In [ ]:
data = fetch_movielens(min_rating=4.0)

In [ ]:
#################
# YOUR CODE HERE
#################
# необходимо определить модель, обучить и вывести метрику precision

In [ ]:
# create app
app = Flask(__name__)

In [ ]:
# API endpoint
@app.route('/')
def recom_for_user():
    user_id = request.args.get('user_id', default = 1, type = int)
    
    n_users, n_items = data['train'].shape
    known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
    scores = model.predict(user_id, np.arange(n_items))
    top_items = data['item_labels'][np.argsort(-scores)][:10]
    
    movie_scores = zip(top_items, scores)
    
    return jsonify(
        movies=[
            {
                "id": i,
                "score": float(s),
            }
            for i, s in movie_scores
        ],
    )

In [ ]:
app.run(host="0.0.0.0", port=5000)